In [ ]:
!pip install datasets==2.2.0

In [ ]:
pip install transformers datasets faiss-cpu sentence-transformers

In [ ]:
from transformers import pipeline
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
import pandas as pd
from datasets import load_dataset
from sentence_transformers import SentenceTransformer, util
import faiss
import numpy as np

ds = load_dataset("allenai/qasper")
import pandas as pd

# extracting split's data
def create_qa_dataframe(ds, split):
    # Lists to store the data
    titles = []
    questions = []
    answers = []
    evidence = []

    # Iterate through the dataset
    for id in range(len(ds[split])):
        title = ds[split]['title'][id]

        # Iterate through questions for each document
        for i in range(len(ds[split][id]['qas']['question'])):
            # Iterate through answers for each question
            print(ds[split][id]['qas']['question'])
            print(ds[split][id]['qas']['answers'][i])

            for j in range(len(ds[split][id]['qas']['answers'][i]['answer'])):
                # Check if there are extractive spans
                if len(ds[split][id]['qas']['answers'][i]['answer'][j]['extractive_spans']) > 0:
                    titles.append(title)
                    evidence.append(ds[split][id]['qas']['answers'][i]['answer'][j]['evidence'])
                    questions.append(ds[split][id]['qas']['question'][i])
                    if len(ds[split][id]['qas']['answers'][i]['answer'][j]['extractive_spans']) > 0:
                        answers.append(ds[split][id]['qas']['answers'][i]['answer'][j]['extractive_spans'][0])
                    else:
                        answers.append(ds[split][id]['qas']['answers'][i]['answer'][j]['extractive_spans'])

    # Create DataFrame
    df = pd.DataFrame({
        'title': titles,
        'question': questions,
        'answer': answers,
        'evidence': evidence
    })

    return df

# Usage example:
split = 'test'
df = create_qa_dataframe(ds, split)
from tqdm import tqdm

# Extracting the whole paper
papers = []
for i in tqdm(range(len(ds[split]))):
  # print(i)
  paper_detail = {}
  paper_detail['title'] = ds[split][i]['title']
  paragraphs = []
  for j in ds[split][i]['full_text']['paragraphs']: # for each section j
    if len(j) > 0: # if the section is not empty
      for i in j: # consider each paragraph in the section
        if len(i) > 0: # and if the paragraph is not empty
          paragraphs.append(i) # append to the list
  paper_detail['paragraphs'] = paragraphs
  papers.append(paper_detail)

complete_df = []
para = pd.DataFrame(papers)
for index, paper_row in para.iterrows():
    for index, df_row in df.iterrows():
        if df_row['title'] == paper_row['title']:
            complete_df.append(paper_row['paragraphs'])

df['paper_content'] = complete_df
df

In [1]:
import pandas as pd
from ast import literal_eval

df = pd.read_csv('/mnt/c/Users/imana/Desktop/Masters/NLP_project/NLP701_QA_Paper/train_set.csv', converters={'paper_content': literal_eval} )
df = df.dropna()
df

,id,title,question,answer,evidence,paper_content,negative_passages
0,0,Minimally Supervised Learning of Affective Eve...,What is the seed lexicon?,seed lexicon consists of positive and negative...,The seed lexicon consists of positive and nega...,[Affective events BIBREF0 are events that typi...,"[""Affective events BIBREF0 are events that typ..."
1,1,Minimally Supervised Learning of Affective Eve...,What are labels available in dataset for super...,negative,Affective events BIBREF0 are events that typic...,[Affective events BIBREF0 are events that typi...,"[""Affective events BIBREF0 are events that typ..."
2,2,Minimally Supervised Learning of Affective Eve...,How large is raw corpus used for training?,100 million sentences,"As a raw corpus, we used a Japanese web corpus...",[Affective events BIBREF0 are events that typi...,"[""Affective events BIBREF0 are events that typ..."
3,3,"PO-EMO: Conceptualization, Annotation, and Mod...",How is the annotation experiment evaluated?,confusion matrices of labels between annotators,We find that Cohen $\kappa $ agreement ranges ...,"[1.1em, 1.1.1em, 1.1.1.1em, Thomas Haider$^{1,...","['1.1em', '1.1.1em', '1.1.1.1em']"
4,4,"PO-EMO: Conceptualization, Annotation, and Mod...",What are the aesthetic emotions formalized?,feelings of suspense experienced in narratives...,To emotionally move readers is considered a pr...,"[1.1em, 1.1.1em, 1.1.1.1em, Thomas Haider$^{1,...","['1.1em', '1.1.1em', '1.1.1.1em']"
...,...,...,...,...,...,...,...
825,1291,Phase transitions in a decentralized graph-bas...,What are three possible phases for language fo...,phase i: $\langle cc \rangle $ increases smoot...,Three clear domains can be noticed in the beha...,[This letter arises from two intriguing questi...,['This letter arises from two intriguing quest...
826,1292,Natural Language Processing with Small Feed-Fo...,What NLP tasks do the authors evaluate feed-fo...,"language identification, part-of-speech taggin...",We experiment with small feed-forward networks...,[Deep and recurrent neural networks with large...,['Deep and recurrent neural networks with larg...
827,1293,Back to the Future -- Sequential Alignment of ...,What are three challenging tasks authors evalu...,paper acceptance prediction,We consider three tasks representing a broad s...,"[As time passes, language usage changes. For e...","[""As time passes, language usage changes. For ..."
828,1294,Analyzing Language Learned by an Active Questi...,What is the difference in findings of Buck et ...,aqa diverges from well structured language in ...,Here we perform a qualitative analysis of this...,[ BIBREF0 propose a reinforcement learning fra...,[' BIBREF0 propose a reinforcement learning fr...


In [ ]:
import nltk
nltk.download('stopwords')
import pandas as pd
import torch
from torch.optim import AdamW
from torch.nn.functional import normalize
from transformers import AutoModel, AutoTokenizer
from torch.utils.data import DataLoader, Dataset
import faiss
from ast import literal_eval
from tqdm import tqdm
import os
import torch

datapath = '/home/iman.alsikaiti/workspace/NLP_project/train_set.csv'
qa_data_df = pd.read_csv(datapath, converters={'negative_passages': literal_eval,
                                                'paper_content': literal_eval})
qa_data_dict = qa_data_df.to_dict('records')

def encode(texts, tokenizer, model):
    # Ensure all inputs are strings
    if not all(isinstance(t, str) for t in texts):
        texts = [str(t) for t in texts]

    # Tokenize and encode
    inputs = tokenizer(
        text = texts,
        padding=True,
        truncation=True,
        return_tensors="pt",
        max_length=256
    )
    outputs = model(**inputs)
    return outputs.last_hidden_state[:, 0, :]  # CLS token embedding

# Load encoders and tokenizers
context_encoder = AutoModel.from_pretrained("/home/iman.alsikaiti/workspace/NLP_project/context_encoder")
context_tokenizer = AutoTokenizer.from_pretrained("/home/iman.alsikaiti/workspace/NLP_project/context_tokenizer")

query_encoder = AutoModel.from_pretrained("/home/iman.alsikaiti/workspace/NLP_project/query_encoder")
query_tokenizer = AutoTokenizer.from_pretrained("/mnt/c/Users/imana/Desktop/Masters/NLP_project/NLP701_QA_Paper/query_tokenizer")

# Step 7: Testing the Retriever
corpus = qa_data_df[qa_data_df['title'] == 'Event Outcome Prediction using Sentiment Analysis and Crowd Wisdom in Microblog Feeds']['paper_content'].values.tolist()
corpus_embeddings = encode(corpus, context_tokenizer, context_encoder).detach().numpy()

# Build FAISS Index
dimension = corpus_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(corpus_embeddings)

query = "Who is the crowd in these experiments?"
query_embedding = encode([query], query_tokenizer, query_encoder).detach().numpy()
distances, indices = index.search(query_embedding, k=3)

# print(f"Top 1 passage: {corpus[indices[0][0]]}")
print(corpus[indices[0][0]][0])

In [ ]:
from transformers import pipeline
import pandas as pd
from typing import Dict, List, Tuple
import torch
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import torch
from torch.optim import AdamW
from torch.nn.functional import normalize
from transformers import AutoModel, AutoTokenizer
from torch.utils.data import DataLoader, Dataset
import faiss
from tqdm import tqdm
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

query_tokenizer = AutoTokenizer.from_pretrained("roberta-base")
context_tokenizer = AutoTokenizer.from_pretrained("roberta-base")
query_encoder = AutoModel.from_pretrained("roberta-base")
context_encoder = AutoModel.from_pretrained("roberta-base")
stop_words = set(stopwords.words('english'))

def remove_stopwords(doc):
    words = doc.split()  # Split the document into words
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)  # Reconstruct the document without stopwords

def dot(K, L):
   if len(K) != len(L):
      return 0

   return sum(i[0] * i[1] for i in zip(K, L))

def encode(texts, tokenizer, model):    
    # Tokenize and encode
    inputs = tokenizer(
        text = texts,
        padding=True,
        truncation=True,
        return_tensors="pt",
        max_length=256
    )
    outputs = model(**inputs)
    return outputs.last_hidden_state[:, 0, :]  # CLS token embedding

class QADataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]


class DocumentRetriever:
    """Class to handle TF-IDF based document retrieval"""

    def __init__(self):
        self.vectorizer = TfidfVectorizer(
            strip_accents='unicode',
            lowercase=True,
            stop_words='english'
        )
        self.doc_embs = None
        self.documents = None
        self.doc_vectors = None

    def fit(self, documents: List[str]) -> None:
        """
        Fit the TF-IDF vectorizer on the documents

        Args:
            documents (List[str]): List of document texts
        """
        self.documents = documents
        self.doc_vectors = self.vectorizer.fit_transform(documents)


    def get_most_relevant(self, query: str, top_k: int = 3) -> List[Tuple[str, float]]:
        """
        Get the most relevant documents for a query

        Args:
            query (str): Query text
            top_k (int): Number of top documents to return

        Returns:
            List[Tuple[str, float]]: List of (document, similarity_score) pairs
        """
        # Transform query using fitted vectorizer
        query_vector = self.vectorizer.transform([query])
        # Calculate cosine similarity between query and all documents

        similarities = cosine_similarity(query_vector, self.doc_vectors).flatten()
        # Get indices of top-k most similar documents
        top_indices = similarities.argsort()[-top_k:][::-1]

        # Return documents and their similarity scores
        return [(self.documents[i], float(similarities[i])) for i in top_indices]

def create_qa_pipeline(model_checkpoint: str = 'ImanAndrea/roberta-finetuned-paperQA') -> pipeline:
    """
    Create a question-answering pipeline using the specified model checkpoint.

    Args:
        model_checkpoint (str): HuggingFace model checkpoint to use

    Returns:
        pipeline: HuggingFace question-answering pipeline
    """
    return pipeline("question-answering", model=model_checkpoint, device=0)

def get_answer_from_context(
    question: str,
    context: str,
    qa_pipeline: pipeline
) -> Dict:
    """
    Get answer for a question given the context using the QA pipeline.

    Args:
        question (str): Question to be answered
        context (str): Context text to extract answer from
        qa_pipeline (pipeline): HuggingFace question-answering pipeline

    Returns:
        Dict: Dictionary containing answer and confidence score
    """
    return qa_pipeline(question=question, context=context)

def process_paper_qa(
    df: pd.DataFrame,
    question: str,
    qa_pipeline: pipeline,
    retriever: DocumentRetriever,
    top_k: int = 3
) -> Dict:
    """
    Process paper QA using TF-IDF retrieval and QA pipeline

    Args:
        df (pd.DataFrame): DataFrame containing paper data
        question (str): Question to be answered
        qa_pipeline (pipeline): HuggingFace question-answering pipeline
        retriever (DocumentRetriever): Document retriever instance
        top_k (int): Number of top paragraphs to consider

    Returns:
        Dict: Answer and metadata
    """
    # Get most relevant paragraphs using TF-IDF
    relevant_docs = retriever.get_most_relevant(question, top_k=top_k)

    # Initialize best answer
    best_answer = None
    best_score = -1
    best_context = None
    best_similarity = -1

    # Try QA on each relevant paragraph
    for doc, similarity in relevant_docs:
        answer = get_answer_from_context(
            question=question,
            context=doc,
            qa_pipeline=qa_pipeline
        )

        # Update best answer if current answer has higher confidence
        if answer['score'] > best_score:
            best_answer = answer['answer']
            best_score = answer['score']
            best_context = doc
            best_similarity = similarity
            
    # print('best answer:', best_answer)

    return {
        'question': question,
        'answer': best_answer,
        'confidence': best_score,
        'context': best_context,
        'context_similarity': best_similarity
    }

# Example usage:
if __name__ == "__main__":
    # Create the QA pipeline
    qa_pipeline = create_qa_pipeline()
    result = []

    # Sample dataframe (based on the image)
    for index, data in tqdm(df.iterrows()):
        #print(data)

        # Initialize and fit the retriever
        retriever = DocumentRetriever()
        retriever.fit(data['paper_content'])

        # Get question from current row
        question = data['question']

        # Get answer
        result.append(process_paper_qa(data, question, qa_pipeline, retriever))

    # Add the evaluation pweaase

2024-12-01 12:09:13.521169: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733040553.584491   12249 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733040553.598625   12249 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-01 12:09:13.807835: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/imanandrea/nltk_data...
[nltk_data]   Package stopwords is

In [8]:
pred_answer = pd.DataFrame(result)
pred_answer = pred_answer.drop_duplicates(subset=['question'])
pred_answer.to_csv("predicted_answers_1.csv")

In [4]:
# ground_truth = pd.DataFrame()
# ground_truth['question'] = df['question']
# ground_truth['answer'] = df['answer']
# ground_truth['context'] = df['evidence']

ground_truth = []
for index, row in df.iterrows():
    ground_truth.append({'question': row['question'], 'answer': row['answer'], 'evidence': row['evidence']})


In [5]:
def compute_f1(prediction, ground_truth):
    # Normalize by removing extra whitespace and lowercasing
    pred_tokens = prediction.strip().lower().split()
    truth_tokens = ground_truth.strip().lower().split()
    
    common_tokens = set(pred_tokens) & set(truth_tokens)
    
    if len(common_tokens) == 0:
        return 0
    
    precision = len(common_tokens) / len(pred_tokens)
    recall = len(common_tokens) / len(truth_tokens)
    
    if precision + recall == 0:
        return 0
    
    f1 = 2 * (precision * recall) / (precision + recall)
    return f1

In [7]:
# Extract the prediction text and ground truth for each sample
predicted_answers = [pred["answer"] for pred in result]
print(predicted_answers)
true_answers = [truth["answer"] for truth in ground_truth]

# Compute F1 scores for each prediction
f1_scores = []
for pred, true in zip(predicted_answers, true_answers):
    f1 = compute_f1(pred, true)
    f1_scores.append(f1)

average_f1 = np.mean(f1_scores)
print(f"Average F1 Score: {average_f1}")

[None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, Non

AttributeError: 'NoneType' object has no attribute 'strip'

In [26]:
def compute_exact_match(prediction, ground_truth):
    return int(prediction.strip() == ground_truth.strip())

em_scores = []
for pred, true in zip(predicted_answers, true_answers):
    em = compute_exact_match(pred, true)
    em_scores.append(em)

# Compute the overall Exact Match score as a percentage
exact_match_percentage = np.mean(em_scores) * 100
print(f"Exact Match Score: {exact_match_percentage}%")

Exact Match Score: 0.24096385542168677%
